In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from hrithika_model import spec_grav_model_newdata
from keras.optimizers import Adam

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
! nvidia-smi

Thu Apr 22 18:17:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.63       Driver Version: 457.63       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX250      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   44C    P8    N/A /  N/A |     64MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
BATCH_SIZE = 32
INPUT_SHAPE = (256, 64, 3)
LEN_TRAIN = 10500
LEN_VAL = 1500

In [5]:
img_data = ImageDataGenerator(
                rescale = 1./255
            )

In [6]:
train_gen = img_data.flow_from_directory(
                'Hrithika_LIGO_data_structured/train',
                target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
                batch_size=BATCH_SIZE,
                class_mode='sparse',
                classes=['bbh','bns','noise'],
                shuffle=True,
                seed=101
            )
val_gen = img_data.flow_from_directory(
                'Hrithika_LIGO_data_structured/val',
                target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
                batch_size=BATCH_SIZE,
                class_mode='sparse',
                classes=['bbh','bns','noise']
            )

Found 10500 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.


In [7]:
model = spec_grav_model_newdata(input_shape=INPUT_SHAPE)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 64, 3)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 62, 32)       896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 31, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 29, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 6, 128)        0     

In [9]:
filepath = './hrithika_specgrav_model_files_LIGO_new_data/spec-grav-model-newdata-{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath=filepath, save_weights_only=False, verbose=1)

In [10]:
opt = Adam(lr=1e-4)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
        train_gen,
        steps_per_epoch=(LEN_TRAIN//BATCH_SIZE),
        epochs=10,
        validation_data=val_gen,
        validation_steps=(LEN_VAL//BATCH_SIZE),
        callbacks=[model_checkpoint]
)

Epoch 1/10
328/328 [==============================] - ETA: 0s - loss: 0.8879 - accuracy: 0.5550
Epoch 00001: saving model to ./hrithika_specgrav_model_files_LIGO_new_data/spec-grav-model-newdata-01-0.72.hdf5
328/328 [==============================] - 158s 481ms/step - loss: 0.8879 - accuracy: 0.5550 - val_loss: 0.6238 - val_accuracy: 0.7242
Epoch 2/10
328/328 [==============================] - ETA: 0s - loss: 0.5917 - accuracy: 0.7487
Epoch 00002: saving model to ./hrithika_specgrav_model_files_LIGO_new_data/spec-grav-model-newdata-02-0.77.hdf5
328/328 [==============================] - 34s 103ms/step - loss: 0.5917 - accuracy: 0.7487 - val_loss: 0.5227 - val_accuracy: 0.7656
Epoch 3/10
328/328 [==============================] - ETA: 0s - loss: 0.5210 - accuracy: 0.7861
Epoch 00003: saving model to ./hrithika_specgrav_model_files_LIGO_new_data/spec-grav-model-newdata-03-0.77.hdf5
328/328 [==============================] - 37s 114ms/step - loss: 0.5210 - accuracy: 0.7861 - val_loss: 0.5

In [18]:
from keras.models import load_model

filepath = './hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath=filepath, save_weights_only=False, verbose=1)

model = load_model('./hrithika_specgrav_model_files_LIGO_new_data/spec-grav-model-newdata-10-0.83.hdf5')

history = model.fit(
        train_gen,
        steps_per_epoch=(LEN_TRAIN//BATCH_SIZE),
        epochs=25,
        validation_data=val_gen,
        validation_steps=(LEN_VAL//BATCH_SIZE),
        callbacks=[model_checkpoint]
)

Epoch 1/25
328/328 [==============================] - ETA: 0s - loss: 0.4027 - accuracy: 0.8407
Epoch 00001: saving model to ./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-01-0.83.hdf5
328/328 [==============================] - 34s 105ms/step - loss: 0.4027 - accuracy: 0.8407 - val_loss: 0.4210 - val_accuracy: 0.8302
Epoch 2/25
328/328 [==============================] - ETA: 0s - loss: 0.3925 - accuracy: 0.8411
Epoch 00002: saving model to ./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-02-0.84.hdf5
328/328 [==============================] - 28s 86ms/step - loss: 0.3925 - accuracy: 0.8411 - val_loss: 0.4120 - val_accuracy: 0.8370
Epoch 3/25
328/328 [==============================] - ETA: 0s - loss: 0.3900 - accuracy: 0.8440
Epoch 00003: saving model to ./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-03-0.83.hdf5
328/328 [==============================] - 28s 85ms/step - lo

Epoch 24/25
328/328 [==============================] - ETA: 0s - loss: 0.1852 - accuracy: 0.9291
Epoch 00024: saving model to ./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-24-0.81.hdf5
328/328 [==============================] - 28s 86ms/step - loss: 0.1852 - accuracy: 0.9291 - val_loss: 0.6129 - val_accuracy: 0.8077
Epoch 25/25
328/328 [==============================] - ETA: 0s - loss: 0.1736 - accuracy: 0.9333
Epoch 00025: saving model to ./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-25-0.80.hdf5
328/328 [==============================] - 28s 86ms/step - loss: 0.1736 - accuracy: 0.9333 - val_loss: 0.6314 - val_accuracy: 0.7969


In [22]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model

model = load_model('./hrithika_specgrav_model_files_LIGO_new_data_run_two/spec-grav-model-newdata-run-two-07-0.83.hdf5')

img = image.load_img(f'F:/SpecGrav/LIGO_detection_data/bbh/L1-GW170104.jpg', target_size=(256, 64))
img = image.img_to_array(img)
img /= 255
img_array = np.array([img])
pred = model.predict(img_array)
print(pred)
index = np.argmax(pred[0])
# print(event_list[index])

[[9.9779809e-01 9.0487156e-04 1.2971479e-03]]
